In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
wiki = load_dataset("wikimedia/wikipedia", "20231101.az")["train"].to_pandas()

In [ ]:
raw_datasets = load_dataset("")

In [ ]:
df = pd.read_parquet("/Users/eljan/Documents/azGPT/data/corpus-raw/chunks.parquet")

In [ ]:
raw_datasets['train']

In [ ]:
df1 = raw_datasets['train'].to_pandas()

In [ ]:
df1 = df1[['text']]
df = df[['text']]

In [ ]:
df = pd.concat([df, df1], ignore_index=True, axis=0)
df.shape

In [ ]:
# shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)
# train, validation split
train_df = df[:int(0.9*len(df))]
val_df = df[int(0.9*len(df)):]

In [ ]:
# convert to huggingface dataset and upload to the hub
from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset = DatasetDict({"train": train_dataset, "val": val_dataset})

In [ ]:

dataset.push_to_hub("eljanmahammadli/wiki_and_chunks")

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

old_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf")

def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["text"]

training_corpus = get_training_corpus()
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 32000)


In [ ]:
tokenizer.save_pretrained("azGPT-LLaMA-tokenizer-perplexity")
tokenizer.push_to_hub("azGPT-perplexity")

In [ ]:
df = raw_datasets['train'].to_pandas()

In [ ]:
from tqdm import tqdm
tqdm.pandas()
df['token_length'] = df['text'].progress_apply(lambda x: len(tokenizer(x)['input_ids']))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df['token_length'].describe()

In [ ]:
df[df['token_length'] < 1024]['token_length'].plot(kind='hist', bins=100)

In [ ]:
df[df['token_length'] < 512].shape[0] / df.shape[0] * 100

In [ ]:
df.shape

In [ ]:
tokenizer.voc

In [ ]:
from transformers import AutoTokenizer
import math

original_vocab_size = len(tokenizer)

# Calculate the padded vocabulary size (e.g., to the next multiple of 8)
def pad_vocab_size_to_multiple(vocab_size, multiple_of):
    return math.ceil(vocab_size / multiple_of) * multiple_of

padded_vocab_size = pad_vocab_size_to_multiple(original_vocab_size, 8)

print(f"Original Vocabulary Size: {original_vocab_size}")
print(f"Padded Vocabulary Size: {padded_vocab_size}")


In [ ]:
ds = load_dataset("")

# train val split the dataset randomly

df = ds['train'].to_pandas()

df = df.sample(frac=1).reset_index(drop=True)

train_df = df[:int(0.9*len(df))]
val_df = df[int(0.9*len(df)):]

# convert back to huggingface dataset
from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset = DatasetDict({"train": train_dataset, "val": val_dataset})